# Final Project DL model Notebook - linear model

#### imports, settings and constants

In [1]:
import theano.gpuarray
import os, sys

from keras.models import Sequential
from keras.layers import BatchNormalization, Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras import backend
## backend.set_image_dim_ordering('th')

## both lines only needed if importing from utils module
# sys.path.insert(1, os.path.join(sys.path[0], '..'))
# from utils.utils import *

current_dir = os.getcwd()
HOME_DIR = current_dir
DATA_DIR = current_dir+'/data/'

# comment out one of the two path options to toggle between sample directory and all data
# path = DATA_HOME_DIR + '/'
path = DATA_DIR + '/sample/'
train_path = path + '/train/'
val_path = path + '/valid/'
test_path = path + '/test/'
results_path = path + '/results/'

batch_size = 64
epochs = 5

Using Theano backend.


#### getting training and validation data in batches

In [2]:
gen=image.ImageDataGenerator()
batches = gen.flow_from_directory(train_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=batch_size)
val_batches = gen.flow_from_directory(val_path, target_size=(224,224), class_mode='categorical', shuffle=False, batch_size=batch_size * 2)
test_batches = gen.flow_from_directory(test_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=batch_size)

Found 1000 images belonging to 10 classes.
Found 200 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


#### and getting the classes, labels and filenames for each batch

In [3]:
trn_classes = batches.classes
val_classes = val_batches.classes
trn_labels = to_categorical(batches.classes)
val_labels = to_categorical(val_batches.classes)
trn_filenames = batches.filenames
val_filenames = val_batches.filenames

#### defining linear model

In [4]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

#### fit_generator() consuming the batches provided by the ImageDataGenerator to fit the model to the data

In [5]:
model.fit_generator(batches,
                    steps_per_epoch=batches.samples//batches.batch_size,
                    validation_data=val_batches,
                    validation_steps=val_batches.samples//val_batches.batch_size,
                    epochs=epochs)

Epoch 1/5
15/15 [==============================] - 23s - loss: 13.2756 - acc: 0.1177 - val_loss: 13.8515 - val_acc: 0.1406
Epoch 2/5
15/15 [==============================] - 20s - loss: 13.8311 - acc: 0.1371 - val_loss: 14.1033 - val_acc: 0.1250
Epoch 3/5
15/15 [==============================] - 29s - loss: 13.9128 - acc: 0.1317 - val_loss: 14.1033 - val_acc: 0.1250
Epoch 4/5
15/15 [==============================] - 20s - loss: 13.9183 - acc: 0.1335 - val_loss: 14.2806 - val_acc: 0.1094
Epoch 5/5
15/15 [==============================] - 21s - loss: 14.3392 - acc: 0.1080 - val_loss: 14.1033 - val_acc: 0.1250


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 3, 224, 224)       12        
_________________________________________________________________
flatten_1 (Flatten)          (None, 150528)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1505290   
Total params: 1,505,302
Trainable params: 1,505,296
Non-trainable params: 6
_________________________________________________________________
